# Why feature learning is better than simple propositionalization

In this notebook we will compare getML to featuretools and tsfresh, both of which open-source libraries for feature engineering. We find that advanced algorithms featured in getML yield significantly better predictions on this dataset. We then discuss why that is.

Summary:

- Prediction type: __Regression model__
- Domain: __Air pollution__
- Prediction target: __pm 2.5 concentration__
- Source data: __Multivariate time series__
- Population size: __41757__

# Background

Many data scientists and AutoML tools use propositionalization methods for feature engineering. These propositionalization methods usually work as follows:

- Generate a large number of hard-coded features,
- Use feature selection to pick a percentage of these features.

By contrast, getML contains approaches for feature learning: Feature learning adapts machine learning approaches such as decision trees or gradient boosting to the problem of extracting features from relational data and time series.

In this notebook, we will benchmark getML against [featuretools](https://www.featuretools.com/) and [tsfresh](https://tsfresh.readthedocs.io/en/latest/). Both of these libaries use propositionalization approaches for feature engineering.

As our example dataset, we use a publicly available dataset on air pollution in Beijing, China (https://archive.ics.uci.edu/ml/datasets/Beijing+PM2.5+Data). The data set has been originally used in the following study:
> Liang, X., Zou, T., Guo, B., Li, S., Zhang, H., Zhang, S., Huang, H. and Chen, S. X. (2015). Assessing Beijing's PM2.5 pollution: severity, weather impact, APEC and winter heating. Proceedings of the Royal Society A, 471, 20150257.

We find that getML significantly outperforms featuretools and tsfresh in terms of predictive accuracy (R-squared of 62.3% vs R-squared of 50.4%). Our findings indicate that getML's feature learning algorithms are better at adapting to data sets and are also more scalable due to their lower memory requirement.

# A web frontend for getML

The getML monitor is a frontend built to support your work with getML. The getML monitor displays information such as the imported data frames, trained pipelines and allows easy data and feature exploration. You can launch the getML monitor [here](http://localhost:1709).

# Analysis

## Table of contents

1. [Loading data](#1.-Loading-data)
2. [Predictive modeling](#2.-Predictive-modeling)
3. [Discussion](#3.-Discussion)
4. [Conclusion](#4.-Conclusion)

We start the analysis with the setup of our session.

In [2]:
import os
from urllib import request

import getml
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from scipy.stats import pearsonr

from utils.load import load_or_retrieve

%matplotlib inline

In [3]:
# NOTE: Due to featuretools's and tsfresh's substantial memory requirements, this notebook will not run on
# try.getml.com when RUN_FEATURETOOLS or RUN_TSFRESH is set.

RUN_FEATURETOOLS = False
RUN_TSFRESH = False

if RUN_FEATURETOOLS:
    from utils import FTTimeSeriesBuilder

if RUN_TSFRESH:
    from utils import TSFreshBuilder

In [4]:
getml.engine.launch()
getml.set_project("air_pollution")

getML engine is already running.

Connected to project 'air_pollution'


## 1. Loading data

### 1.1 Download from source

Downloading the raw data from the  UCI Machine Learning Repository into a prediction ready format takes time. To get to the getML model building as fast as possible, we prepared the data for you and excluded the code from this notebook.

In [5]:
data = getml.datasets.load_air_pollution()


Loading population...
 100% |██████████| [elapsed: 00:00, remaining: 00:00]          


## 2. Predictive modeling


### 2.1 Pipeline 1: Complex features, 7 days

First, we spilt our data. We introduce a [simple, time-based split](https://docs.getml.com/latest/api/split/getml.data.split.time.html) and use all data until 2013-12-31 for training and everything starting from 2014-01-01 for testing.

In [6]:
split = getml.data.split.time(
    population=data, time_stamp="date", test=getml.data.time.datetime(2014, 1, 1)
)

split

,
0,train
1,train
2,train
3,train
4,train
,...


For our first experiment, we will learn complex features and allow a memory of up to seven days. That means at every given point in time, the algorithm is allowed to back seven days into the past.

In [7]:
time_series1 = getml.data.TimeSeries(
    population=data,
    alias="population",
    split=split,
    time_stamps="date",
    memory=getml.data.time.days(7),
)

time_series1

data model

  population:
    columns:
    - DEWP: numerical
    - TEMP: numerical
    - PRES: numerical
    - Iws: numerical
    - Is: numerical
    - ...

    joins:
    - right: 'population'
      time_stamps: (population.date, population.date)
      relationship: 'many-to-many'
      memory: 604800.0
      lagged_targets: False

  population:
    columns:
    - DEWP: numerical
    - TEMP: numerical
    - PRES: numerical
    - Iws: numerical
    - Is: numerical
    - ...


container

  population
      subset   name          rows   type
  0   test     population    8661   View
  1   train    population   33096   View

  peripheral
      name          rows   type     
  0   population   41757   DataFrame

In [8]:
relmt = getml.feature_learning.RelMT(
    num_features=10,
    loss_function=getml.feature_learning.loss_functions.SquareLoss,
    seed=4367,
    num_threads=1,
)

predictor = getml.predictors.XGBoostRegressor(n_jobs=1)

pipe1 = getml.pipeline.Pipeline(
    tags=["getML: RelMT", "memory: 7d", "complex features"],
    data_model=time_series1.data_model,
    feature_learners=[relmt],
    predictors=[predictor],
)

pipe1

Pipeline(data_model='population',
         feature_learners=['RelMT'],
         feature_selectors=[],
         include_categorical=False,
         loss_function='SquareLoss',
         peripheral=['population'],
         predictors=['XGBoostRegressor'],
         preprocessors=[],
         share_selected_features=0.5,
         tags=['getML: RelMT', 'memory: 7d', 'complex features'])

It is good practice to always check your data model first, even though `check(...)` is also called by `fit(...)`. That enables us to make last-minute changes.

In [9]:
pipe1.check(time_series1.train)

Checking data model...
Staging... 100% |██████████| [elapsed: 00:00, remaining: 00:00]          
Checking... 100% |██████████| [elapsed: 00:02, remaining: 00:00]          

OK.


We now fit our data on the training set and evaluate our findings, both, in-sample and out-of-sample.

In [10]:
pipe1.fit(time_series1.train)

Checking data model...
Staging... 100% |██████████| [elapsed: 00:00, remaining: 00:00]          

OK.
Staging... 100% |██████████| [elapsed: 00:00, remaining: 00:00]          
RelMT: Training features... 100% |██████████| [elapsed: 01:17, remaining: 00:00]          
RelMT: Building features... 100% |██████████| [elapsed: 00:15, remaining: 00:00]          
XGBoost: Training as predictor... 100% |██████████| [elapsed: 00:02, remaining: 00:00]          

Trained pipeline.
Time taken: 0h:1m:34.611977



Pipeline(data_model='population',
         feature_learners=['RelMT'],
         feature_selectors=[],
         include_categorical=False,
         loss_function='SquareLoss',
         peripheral=['population'],
         predictors=['XGBoostRegressor'],
         preprocessors=[],
         share_selected_features=0.5,
         tags=['getML: RelMT', 'memory: 7d', 'complex features', 'container-P61D6i'])

In [11]:
pipe1.score(time_series1.test)

Staging... 100% |██████████| [elapsed: 00:00, remaining: 00:00]          
Preprocessing... 100% |██████████| [elapsed: 00:00, remaining: 00:00]          
RelMT: Building features... 100% |██████████| [elapsed: 00:04, remaining: 00:00]          



,date time,set used,target,mae,rmse,rsquared
0,2023-07-29 20:22:50,train,pm2.5,35.1664,50.9038,0.6925
1,2023-07-29 20:22:54,test,pm2.5,39.6596,57.5014,0.6306


### 2.2 Pipeline 2: Complex features, 1 day

In [12]:
time_series2 = getml.data.TimeSeries(
    population=data,
    alias="population",
    split=split,
    time_stamps="date",
    memory=getml.data.time.days(1),
)

time_series2

data model

  population:
    columns:
    - DEWP: numerical
    - TEMP: numerical
    - PRES: numerical
    - Iws: numerical
    - Is: numerical
    - ...

    joins:
    - right: 'population'
      time_stamps: (population.date, population.date)
      relationship: 'many-to-many'
      memory: 86400.0
      lagged_targets: False

  population:
    columns:
    - DEWP: numerical
    - TEMP: numerical
    - PRES: numerical
    - Iws: numerical
    - Is: numerical
    - ...


container

  population
      subset   name          rows   type
  0   test     population    8661   View
  1   train    population   33096   View

  peripheral
      name          rows   type     
  0   population   41757   DataFrame

In [13]:
relmt = getml.feature_learning.RelMT(
    num_features=10,
    loss_function=getml.feature_learning.loss_functions.SquareLoss,
    seed=4367,
    num_threads=1,
)

predictor = getml.predictors.XGBoostRegressor(n_jobs=1)

pipe2 = getml.pipeline.Pipeline(
    tags=["getML: RelMT", "memory: 1d", "complex features"],
    data_model=time_series2.data_model,
    feature_learners=[relmt],
    predictors=[predictor],
)

pipe2

Pipeline(data_model='population',
         feature_learners=['RelMT'],
         feature_selectors=[],
         include_categorical=False,
         loss_function='SquareLoss',
         peripheral=['population'],
         predictors=['XGBoostRegressor'],
         preprocessors=[],
         share_selected_features=0.5,
         tags=['getML: RelMT', 'memory: 1d', 'complex features'])

In [14]:
pipe2.check(time_series2.train)

Checking data model...
Staging... 100% |██████████| [elapsed: 00:00, remaining: 00:00]          
Checking... 100% |██████████| [elapsed: 00:02, remaining: 00:00]          

OK.


In [15]:
pipe2.fit(time_series2.train)

Checking data model...
Staging... 100% |██████████| [elapsed: 00:00, remaining: 00:00]          

OK.
Staging... 100% |██████████| [elapsed: 00:00, remaining: 00:00]          
RelMT: Training features... 100% |██████████| [elapsed: 00:18, remaining: 00:00]          
RelMT: Building features... 100% |██████████| [elapsed: 00:02, remaining: 00:00]          
XGBoost: Training as predictor... 100% |██████████| [elapsed: 00:02, remaining: 00:00]          

Trained pipeline.
Time taken: 0h:0m:22.307469



Pipeline(data_model='population',
         feature_learners=['RelMT'],
         feature_selectors=[],
         include_categorical=False,
         loss_function='SquareLoss',
         peripheral=['population'],
         predictors=['XGBoostRegressor'],
         preprocessors=[],
         share_selected_features=0.5,
         tags=['getML: RelMT', 'memory: 1d', 'complex features', 'container-6mTZst'])

In [16]:
pipe2.score(time_series2.test)

Staging... 100% |██████████| [elapsed: 00:00, remaining: 00:00]          
Preprocessing... 100% |██████████| [elapsed: 00:00, remaining: 00:00]          
RelMT: Building features... 100% |██████████| [elapsed: 00:01, remaining: 00:00]          



,date time,set used,target,mae,rmse,rsquared
0,2023-07-29 20:23:19,train,pm2.5,38.1593,55.3541,0.6366
1,2023-07-29 20:23:19,test,pm2.5,47.5451,66.9418,0.4901


### 2.3 Pipeline 3: Simple features, 7 days

For our third experiment, we will learn simple features and allow a memory of up to seven days.

In [17]:
time_series3 = getml.data.TimeSeries(
    population=data,
    alias="population",
    split=split,
    time_stamps="date",
    memory=getml.data.time.days(7),
)

time_series3

data model

  population:
    columns:
    - DEWP: numerical
    - TEMP: numerical
    - PRES: numerical
    - Iws: numerical
    - Is: numerical
    - ...

    joins:
    - right: 'population'
      time_stamps: (population.date, population.date)
      relationship: 'many-to-many'
      memory: 604800.0
      lagged_targets: False

  population:
    columns:
    - DEWP: numerical
    - TEMP: numerical
    - PRES: numerical
    - Iws: numerical
    - Is: numerical
    - ...


container

  population
      subset   name          rows   type
  0   test     population    8661   View
  1   train    population   33096   View

  peripheral
      name          rows   type     
  0   population   41757   DataFrame

In [18]:
fast_prop = getml.feature_learning.FastProp(
    loss_function=getml.feature_learning.loss_functions.SquareLoss,
    num_threads=1,
    aggregation=getml.feature_learning.FastProp.agg_sets.All,
)

predictor = getml.predictors.XGBoostRegressor(n_jobs=1)

pipe3 = getml.pipeline.Pipeline(
    tags=["getML: FastProp", "memory: 7d", "simple features"],
    data_model=time_series3.data_model,
    feature_learners=[fast_prop],
    predictors=[predictor],
)

pipe3

Pipeline(data_model='population',
         feature_learners=['FastProp'],
         feature_selectors=[],
         include_categorical=False,
         loss_function='SquareLoss',
         peripheral=['population'],
         predictors=['XGBoostRegressor'],
         preprocessors=[],
         share_selected_features=0.5,
         tags=['getML: FastProp', 'memory: 7d', 'simple features'])

In [19]:
pipe3.check(time_series3.train)

Checking data model...
Staging... 100% |██████████| [elapsed: 00:00, remaining: 00:00]          
Checking... 100% |██████████| [elapsed: 00:02, remaining: 00:00]          

OK.


In [20]:
pipe3.fit(time_series3.train)

Checking data model...
Staging... 100% |██████████| [elapsed: 00:00, remaining: 00:00]          

OK.
Staging... 100% |██████████| [elapsed: 00:00, remaining: 00:00]          
FastProp: Trying 378 features... 100% |██████████| [elapsed: 00:18, remaining: 00:00]          
FastProp: Building features... 100% |██████████| [elapsed: 00:10, remaining: 00:00]          
XGBoost: Training as predictor... 100% |██████████| [elapsed: 00:17, remaining: 00:00]          

Trained pipeline.
Time taken: 0h:0m:45.150319



Pipeline(data_model='population',
         feature_learners=['FastProp'],
         feature_selectors=[],
         include_categorical=False,
         loss_function='SquareLoss',
         peripheral=['population'],
         predictors=['XGBoostRegressor'],
         preprocessors=[],
         share_selected_features=0.5,
         tags=['getML: FastProp', 'memory: 7d', 'simple features', 'container-x6DPMz'])

In [21]:
pipe3.score(time_series3.test)

Staging... 100% |██████████| [elapsed: 00:00, remaining: 00:00]          
Preprocessing... 100% |██████████| [elapsed: 00:00, remaining: 00:00]          
FastProp: Building features... 100% |██████████| [elapsed: 00:03, remaining: 00:00]          



,date time,set used,target,mae,rmse,rsquared
0,2023-07-29 20:24:07,train,pm2.5,35.9677,50.7711,0.7036
1,2023-07-29 20:24:10,test,pm2.5,45.4586,62.6197,0.5617


### 2.4 Pipeline 4: Simple features, 1 day

For our fourth experiment, we will learn simple features and allow a memory of up to one day.

In [22]:
time_series4 = getml.data.TimeSeries(
    population=data,
    alias="population",
    split=split,
    time_stamps="date",
    memory=getml.data.time.days(1),
)

time_series4

data model

  population:
    columns:
    - DEWP: numerical
    - TEMP: numerical
    - PRES: numerical
    - Iws: numerical
    - Is: numerical
    - ...

    joins:
    - right: 'population'
      time_stamps: (population.date, population.date)
      relationship: 'many-to-many'
      memory: 86400.0
      lagged_targets: False

  population:
    columns:
    - DEWP: numerical
    - TEMP: numerical
    - PRES: numerical
    - Iws: numerical
    - Is: numerical
    - ...


container

  population
      subset   name          rows   type
  0   test     population    8661   View
  1   train    population   33096   View

  peripheral
      name          rows   type     
  0   population   41757   DataFrame

In [23]:
fast_prop = getml.feature_learning.FastProp(
    loss_function=getml.feature_learning.loss_functions.SquareLoss,
    num_threads=1,
    aggregation=getml.feature_learning.FastProp.agg_sets.All,
)

predictor = getml.predictors.XGBoostRegressor(n_jobs=1)

pipe4 = getml.pipeline.Pipeline(
    tags=["getML: FastProp", "memory: 1d", "simple features"],
    data_model=time_series4.data_model,
    feature_learners=[fast_prop],
    predictors=[predictor],
)

pipe4

Pipeline(data_model='population',
         feature_learners=['FastProp'],
         feature_selectors=[],
         include_categorical=False,
         loss_function='SquareLoss',
         peripheral=['population'],
         predictors=['XGBoostRegressor'],
         preprocessors=[],
         share_selected_features=0.5,
         tags=['getML: FastProp', 'memory: 1d', 'simple features'])

In [24]:
pipe4.check(time_series4.train)

Checking data model...
Staging... 100% |██████████| [elapsed: 00:00, remaining: 00:00]          
Checking... 100% |██████████| [elapsed: 00:02, remaining: 00:00]          

OK.


In [25]:
pipe4.fit(time_series4.train)

Checking data model...
Staging... 100% |██████████| [elapsed: 00:00, remaining: 00:00]          

OK.
Staging... 100% |██████████| [elapsed: 00:00, remaining: 00:00]          
FastProp: Trying 378 features... 100% |██████████| [elapsed: 00:04, remaining: 00:00]          
FastProp: Building features... 100% |██████████| [elapsed: 00:02, remaining: 00:00]          
XGBoost: Training as predictor... 100% |██████████| [elapsed: 00:17, remaining: 00:00]          

Trained pipeline.
Time taken: 0h:0m:23.192945



Pipeline(data_model='population',
         feature_learners=['FastProp'],
         feature_selectors=[],
         include_categorical=False,
         loss_function='SquareLoss',
         peripheral=['population'],
         predictors=['XGBoostRegressor'],
         preprocessors=[],
         share_selected_features=0.5,
         tags=['getML: FastProp', 'memory: 1d', 'simple features', 'container-tcwIPH'])

In [26]:
pipe4.score(time_series4.test)

Staging... 100% |██████████| [elapsed: 00:00, remaining: 00:00]          
Preprocessing... 100% |██████████| [elapsed: 00:00, remaining: 00:00]          
FastProp: Building features... 100% |██████████| [elapsed: 00:01, remaining: 00:00]          



,date time,set used,target,mae,rmse,rsquared
0,2023-07-29 20:24:35,train,pm2.5,38.3028,55.2472,0.6438
1,2023-07-29 20:24:36,test,pm2.5,44.2526,63.4191,0.5462


### 2.5 Using featuretools

To make things a bit easier, we have written high-level wrappers around featuretools and tsfresh which we placed in a separate module (`utils`).

In [27]:
data_train_pandas = time_series1.train.population.to_pandas()
data_test_pandas = time_series1.test.population.to_pandas()

tsfresh and featuretools require the time series to have ids. Since there is only a single time series, that series has the same id.

In [28]:
data_train_pandas["id"] = 1
data_test_pandas["id"] = 1

In [29]:
if RUN_FEATURETOOLS:
    ft_builder = FTTimeSeriesBuilder(
        num_features=200,
        horizon=pd.Timedelta(days=0),
        memory=pd.Timedelta(days=1),
        column_id="id",
        time_stamp="date",
        target="pm2.5",
    )
    #
    featuretools_training = ft_builder.fit(data_train_pandas)
    featuretools_test = ft_builder.transform(data_test_pandas)

    data_featuretools_training = getml.data.DataFrame.from_pandas(
        featuretools_training, name="featuretools_training"
    )
    data_featuretools_test = getml.data.DataFrame.from_pandas(
        featuretools_test, name="featuretools_test"
    )

In [30]:
if not RUN_FEATURETOOLS:
    data_featuretools_training = load_or_retrieve(
        "https://static.getml.com/datasets/air_pollution/featuretools/featuretools_training.csv"
    )
    data_featuretools_test = load_or_retrieve(
        "https://static.getml.com/datasets/air_pollution/featuretools/featuretools_test.csv"
    )

 100% |██████████| [elapsed: 00:02, remaining: 00:00]          

 100% |██████████| [elapsed: 00:01, remaining: 00:00]          



In [31]:
def set_roles_featuretools(df):
    df.set_role(["date"], getml.data.roles.time_stamp)
    df.set_role(["pm2.5"], getml.data.roles.target)
    df.set_role(["date"], getml.data.roles.time_stamp)
    df.set_role(df.roles.unused, getml.data.roles.numerical)
    df.set_role(["id"], getml.data.roles.unused_float)
    return df


df_featuretools_training = set_roles_featuretools(data_featuretools_training)
df_featuretools_test = set_roles_featuretools(data_featuretools_test)

In [32]:
predictor = getml.predictors.XGBoostRegressor()

pipe5 = getml.pipeline.Pipeline(
    tags=["featuretools", "memory: 1d", "simple features"], predictors=[predictor]
)

pipe5

Pipeline(data_model='population',
         feature_learners=[],
         feature_selectors=[],
         include_categorical=False,
         loss_function='SquareLoss',
         peripheral=[],
         predictors=['XGBoostRegressor'],
         preprocessors=[],
         share_selected_features=0.5,
         tags=['featuretools', 'memory: 1d', 'simple features'])

In [33]:
pipe5.check(df_featuretools_training)

Checking data model...
Staging... 100% |██████████| [elapsed: 00:00, remaining: 00:00]          
Checking... 100% |██████████| [elapsed: 00:00, remaining: 00:00]          

OK.


In [34]:
pipe5.fit(df_featuretools_training)

Checking data model...
Staging... 100% |██████████| [elapsed: 00:00, remaining: 00:00]          

OK.
Staging... 100% |██████████| [elapsed: 00:00, remaining: 00:00]          
XGBoost: Training as predictor... 100% |██████████| [elapsed: 00:08, remaining: 00:00]          

Trained pipeline.
Time taken: 0h:0m:8.059616



Pipeline(data_model='population',
         feature_learners=[],
         feature_selectors=[],
         include_categorical=False,
         loss_function='SquareLoss',
         peripheral=[],
         predictors=['XGBoostRegressor'],
         preprocessors=[],
         share_selected_features=0.5,
         tags=['featuretools', 'memory: 1d', 'simple features'])

In [35]:
pipe5.score(df_featuretools_test)

Staging... 100% |██████████| [elapsed: 00:00, remaining: 00:00]          
Preprocessing... 100% |██████████| [elapsed: 00:00, remaining: 00:00]          



,date time,set used,target,mae,rmse,rsquared
0,2023-07-29 20:24:49,featuretools_training,pm2.5,38.0455,54.4693,0.6567
1,2023-07-29 20:24:49,featuretools_test,pm2.5,45.3084,64.2717,0.5373


### 2.6 Using tsfresh

Next, we construct features with tsfresh. tsfresh is based on pandas and rely\ies on explicit copies for meny operations. This leads to an excessive memory consumption that renders tsfresh nearly unusable for real-world scenarios. Remeber, this is a relatively small data set.

To limit the memory consumption, we undertake the following steps:

- We limit ourselves to a memory of 1 day from any point in time. This is necessary, because tsfresh duplicates records for every time stamp. That means that looking back 7 days instead of one day, the memory consumption would be  seven times as high.
- We extract only tsfresh's `MinimalFCParameters` and `IndexBasedFCParameters` (the latter is a superset of `TimeBasedFCParameters`).

In order to make sure that tsfresh's features can be compared to getML's features, we also do the following:

- We apply tsfresh's built-in feature selection algorithm.
- Of the remaining features, we only keep the 40 features most correlated with the target (in terms of the absolute value of the correlation).
- We add the original columns as additional features.


In [36]:
data_train_pandas

,DEWP,TEMP,PRES,Iws,Is,Ir,pm2.5,date,id
0,-16.0,-4.0,1020.0,1.79,0.0,0.0,129.0,2010-01-02 00:00:00,1
1,-15.0,-4.0,1020.0,2.68,0.0,0.0,148.0,2010-01-02 01:00:00,1
2,-11.0,-5.0,1021.0,3.57,0.0,0.0,159.0,2010-01-02 02:00:00,1
3,-7.0,-5.0,1022.0,5.36,1.0,0.0,181.0,2010-01-02 03:00:00,1
4,-7.0,-5.0,1022.0,6.25,2.0,0.0,138.0,2010-01-02 04:00:00,1
...,...,...,...,...,...,...,...,...,...
33091,-19.0,7.0,1013.0,114.87,0.0,0.0,22.0,2013-12-31 19:00:00,1
33092,-21.0,7.0,1014.0,119.79,0.0,0.0,18.0,2013-12-31 20:00:00,1
33093,-21.0,7.0,1014.0,125.60,0.0,0.0,23.0,2013-12-31 21:00:00,1
33094,-21.0,6.0,1014.0,130.52,0.0,0.0,20.0,2013-12-31 22:00:00,1


In [37]:
if RUN_TSFRESH:
    tsfresh_builder = TSFreshBuilder(
        num_features=200, memory=24, column_id="id", time_stamp="date", target="pm2.5"
    )
    #
    tsfresh_training = tsfresh_builder.fit(data_train_pandas)
    tsfresh_test = tsfresh_builder.transform(data_test_pandas)
    #
    data_tsfresh_training = getml.data.DataFrame.from_pandas(
        tsfresh_training, name="tsfresh_training"
    )
    data_tsfresh_test = getml.data.DataFrame.from_pandas(
        tsfresh_test, name="tsfresh_test"
    )

tsfresh does not contain built-in machine learning algorithms. In order to ensure a fair comparison, we use the exact same machine learning algorithm we have also used for getML: An XGBoost regressor with all hyperparameters set to their default values.

In order to do so, we load the tsfresh features into the getML engine.

In [38]:
if not RUN_TSFRESH:
    data_tsfresh_training = load_or_retrieve(
        "https://static.getml.com/datasets/air_pollution/tsfresh/tsfresh_training.csv"
    )
    data_tsfresh_test = load_or_retrieve(
        "https://static.getml.com/datasets/air_pollution/tsfresh/tsfresh_test.csv"
    )

 100% |██████████| [elapsed: 00:02, remaining: 00:00]          

 100% |██████████| [elapsed: 00:01, remaining: 00:00]          



As usual, we need to set roles:

In [39]:
def set_roles_tsfresh(df):
    df.set_role(["date"], getml.data.roles.time_stamp)
    df.set_role(["pm2.5"], getml.data.roles.target)
    df.set_role(["date"], getml.data.roles.time_stamp)
    df.set_role(df.roles.unused, getml.data.roles.numerical)
    df.set_role(["id"], getml.data.roles.unused_float)
    return df


df_tsfresh_training = set_roles_tsfresh(data_tsfresh_training)
df_tsfresh_test = set_roles_tsfresh(data_tsfresh_test)

In this case, our pipeline is very simple. It only consists of a single XGBoostRegressor.

In [40]:
predictor = getml.predictors.XGBoostRegressor()

pipe6 = getml.pipeline.Pipeline(
    tags=["tsfresh", "memory: 1d", "simple features"], predictors=[predictor]
)

pipe6

Pipeline(data_model='population',
         feature_learners=[],
         feature_selectors=[],
         include_categorical=False,
         loss_function='SquareLoss',
         peripheral=[],
         predictors=['XGBoostRegressor'],
         preprocessors=[],
         share_selected_features=0.5,
         tags=['tsfresh', 'memory: 1d', 'simple features'])

In [41]:
pipe6.check(df_tsfresh_training)

Checking data model...
Staging... 100% |██████████| [elapsed: 00:00, remaining: 00:00]          
Checking... 100% |██████████| [elapsed: 00:00, remaining: 00:00]          

OK.


In [42]:
pipe6.fit(df_tsfresh_training)

Checking data model...
Staging... 100% |██████████| [elapsed: 00:00, remaining: 00:00]          

OK.
Staging... 100% |██████████| [elapsed: 00:00, remaining: 00:00]          
XGBoost: Training as predictor... 100% |██████████| [elapsed: 00:06, remaining: 00:00]          

Trained pipeline.
Time taken: 0h:0m:5.743793



Pipeline(data_model='population',
         feature_learners=[],
         feature_selectors=[],
         include_categorical=False,
         loss_function='SquareLoss',
         peripheral=[],
         predictors=['XGBoostRegressor'],
         preprocessors=[],
         share_selected_features=0.5,
         tags=['tsfresh', 'memory: 1d', 'simple features'])

In [43]:
pipe6.score(df_tsfresh_test)

Staging... 100% |██████████| [elapsed: 00:00, remaining: 00:00]          
Preprocessing... 100% |██████████| [elapsed: 00:00, remaining: 00:00]          



,date time,set used,target,mae,rmse,rsquared
0,2023-07-29 20:24:59,tsfresh_training,pm2.5,40.8062,57.7874,0.6106
1,2023-07-29 20:24:59,tsfresh_test,pm2.5,46.698,65.9163,0.5105


In [44]:
pipe1.features

,target,name,correlation,importance
0,pm2.5,feature_1_1,0.7269,0.18463717
1,pm2.5,feature_1_2,0.7046,0.11726964
2,pm2.5,feature_1_3,0.7158,0.08975971
3,pm2.5,feature_1_4,0.6811,0.01235796
4,pm2.5,feature_1_5,0.7363,0.27688485
,...,...,...,...
11,pm2.5,temp,-0.2112,0.00403082
12,pm2.5,pres,0.0811,0.00672836
13,pm2.5,iws,-0.2166,0.00111994
14,pm2.5,is,0.0045,0.00006808


### 2.7 Studying features

In [45]:
pipe1.features.sort(by="importances")[0].sql

```sql
DROP TABLE IF EXISTS "FEATURE_1_5";

CREATE TABLE "FEATURE_1_5" AS
SELECT SUM( 
    CASE
        WHEN ( t2."iws" > 2.996864 ) AND ( t1."date" - t2."date" > 111439.618138 ) THEN COALESCE( t1."dewp" - 1.514736211828887, 0.0 ) * 0.03601656941447697 + COALESCE( t1."temp" - 11.89228926884439, 0.0 ) * -0.04267662254391098 + COALESCE( t1."is" - 0.06612999800412481, 0.0 ) * -0.04725594483043079 + COALESCE( t1."ir" - 0.2116958286208502, 0.0 ) * -0.06321612761647741 + COALESCE( t1."pres" - 1016.466458208569, 0.0 ) * -0.01400320396717386 + COALESCE( t1."iws" - 25.06146098064165, 0.0 ) * -0.001201539412029254 + COALESCE( t1."date" - 1326377672.037789, 0.0 ) * -1.917475955539329e-07 + COALESCE( t2."dewp" - 1.668379064426448, 0.0 ) * 0.001731759296175797 + COALESCE( t2."is" - 0.06086063096820984, 0.0 ) * 0.06061734513927545 + COALESCE( t2."ir" - 0.2111990813489665, 0.0 ) * -0.05403701995482931 + COALESCE( t2."temp" - 12.06001450501632, 0.0 ) * -0.007673283726265613 + COALESCE( t2."pres" - 1016.398404448205, 0.0 ) * 0.002389770593897226 + COALESCE( t2."iws" - 25.00605463556588, 0.0 ) * 0.0006834211951335751 + COALESCE( t2."date" - 1326341256.690439, 0.0 ) * 1.915110430186805e-07 + -3.8304196950085599e-02
        WHEN ( t2."iws" > 2.996864 ) AND ( t1."date" - t2."date" <= 111439.618138 OR t1."date" IS NULL OR t2."date" IS NULL ) THEN COALESCE( t1."dewp" - 1.514736211828887, 0.0 ) * -0.09710953310710296 + COALESCE( t1."temp" - 11.89228926884439, 0.0 ) * 0.1898035546735945 + COALESCE( t1."is" - 0.06612999800412481, 0.0 ) * 0.2216653258331122 + COALESCE( t1."ir" - 0.2116958286208502, 0.0 ) * 0.2229778651779893 + COALESCE( t1."pres" - 1016.466458208569, 0.0 ) * 0.01372329536668545 + COALESCE( t1."iws" - 25.06146098064165, 0.0 ) * 0.008757819241551553 + COALESCE( t1."date" - 1326377672.037789, 0.0 ) * 3.828783706345967e-05 + COALESCE( t2."dewp" - 1.668379064426448, 0.0 ) * 0.01390967635485115 + COALESCE( t2."is" - 0.06086063096820984, 0.0 ) * -0.05439808277091971 + COALESCE( t2."ir" - 0.2111990813489665, 0.0 ) * -0.1708184598905272 + COALESCE( t2."temp" - 12.06001450501632, 0.0 ) * 0.04153068241839334 + COALESCE( t2."pres" - 1016.398404448205, 0.0 ) * 0.06692208551716841 + COALESCE( t2."iws" - 25.00605463556588, 0.0 ) * 0.0003737881469657851 + COALESCE( t2."date" - 1326341256.690439, 0.0 ) * -3.828541810436263e-05 + -2.6193154405913490e+00
        WHEN ( t2."iws" <= 2.996864 OR t2."iws" IS NULL ) AND ( t1."dewp" > 11.000000 ) THEN COALESCE( t1."dewp" - 1.514736211828887, 0.0 ) * 0.1393137946139917 + COALESCE( t1."temp" - 11.89228926884439, 0.0 ) * -0.01214102611962978 + COALESCE( t1."is" - 0.06612999800412481, 0.0 ) * -0.1338390070730758 + COALESCE( t1."ir" - 0.2116958286208502, 0.0 ) * -0.01622585948031095 + COALESCE( t1."pres" - 1016.466458208569, 0.0 ) * 0.001699285808609624 + COALESCE( t1."iws" - 25.06146098064165, 0.0 ) * 0.00337179994237104 + COALESCE( t1."date" - 1326377672.037789, 0.0 ) * -1.911896491745432e-07 + COALESCE( t2."dewp" - 1.668379064426448, 0.0 ) * -0.06644334043226796 + COALESCE( t2."is" - 0.06086063096820984, 0.0 ) * -0.1417763489022403 + COALESCE( t2."ir" - 0.2111990813489665, 0.0 ) * -0.1305274027711076 + COALESCE( t2."temp" - 12.06001450501632, 0.0 ) * -0.133748145181138 + COALESCE( t2."pres" - 1016.398404448205, 0.0 ) * -0.01591750330515081 + COALESCE( t2."iws" - 25.00605463556588, 0.0 ) * 0.05266241808719854 + COALESCE( t2."date" - 1326341256.690439, 0.0 ) * 1.855002102053967e-07 + 1.4765736905119990e+00
        WHEN ( t2."iws" <= 2.996864 OR t2."iws" IS NULL ) AND ( t1."dewp" <= 11.000000 OR t1."dewp" IS NULL ) THEN COALESCE( t1."dewp" - 1.514736211828887, 0.0 ) * 0.0463861265460101 + COALESCE( t1."temp" - 11.89228926884439, 0.0 ) * -0.026165920606117 + COALESCE( t1."is" - 0.06612999800412481, 0.0 ) * -0.04279224372354885 + COALESCE( t1."ir" - 0.2116958286208502, 0.0 ) * -0.02539472205638761 + COALESCE( t1."pres" - 1016.466458208569, 0.0 ) * -0.02717553581795197 + COALESCE( t1."iws" - 25.06146098064165, 0.0 ) * -0.002779614155177693 + COALESCE( t1."date" - 1326377672.037789, 0.0 ) * -1.127852679974305e-06 + COALESCE( t2."dewp" - 1.668379064426448, 0.0 ) * -0.009599325701739291 + COALESCE( t2."is" - 0.06086063096820984, 0.0 ) * -0.244032413107839 + COALESCE( t2."ir" - 0.2111990813489665, 0.0 ) * -0.119801765682448 + COALESCE( t2."temp" - 12.06001450501632, 0.0 ) * -0.0723450466792257 + COALESCE( t2."pres" - 1016.398404448205, 0.0 ) * -0.006849322683184702 + COALESCE( t2."iws" - 25.00605463556588, 0.0 ) * -0.4129622547328788 + COALESCE( t2."date" - 1326341256.690439, 0.0 ) * 1.129731347718475e-06 + -9.4331292048547279e+00
        ELSE NULL
    END
) AS "feature_1_5",
       t1.rowid AS rownum
FROM "POPULATION__STAGING_TABLE_1" t1
INNER JOIN "POPULATION__STAGING_TABLE_2" t2
ON 1 = 1
WHERE t2."date" <= t1."date"
AND ( t2."date__7_000000_days" > t1."date" OR t2."date__7_000000_days" IS NULL )
GROUP BY t1.rowid;
```

This is a typical [RelMT](https://docs.getml.com/latest/user_guide/feature_engineering/feature_engineering.html#relmt) feature, where the aggregation (`SUM` in this case) is applied conditionally – the conditions are learned by `RelMT` – to a set of linear models, whose weights are, again, learned by `RelMT`.

### 2.8 Productionization

It is possible to productionize the pipeline by transpiling the features into production-ready SQL code. Please also refer to getML's `sqlite3` module.

In [46]:
# Creates a folder named air_pollution_pipeline containing the SQL code
pipe1.features.to_sql().save("air_pollution_pipeline")

## 3. Discussion

We have seen that getML outperforms tsfresh by more than 10 percentage points in terms of R-squared. We now want to analyze why that is.

There are two possible hypotheses:

- getML outperforms featuretools and tsfresh, because it using feature learning and is able to produce more complex features
- getML outperforms featuretools and tsfresh, because it makes better use of memory and is able to look back further.

Let's summarize our findings:

In [47]:
pipes = [pipe1, pipe2, pipe3, pipe4, pipe5, pipe6]

comparison = pd.DataFrame(
    dict(
        tool=[pipe.tags[0] for pipe in pipes],
        memory=[pipe.tags[1].split()[1] for pipe in pipes],
        feature_complexity=[pipe.tags[2].split()[0] for pipe in pipes],
        rsquared=[f"{pipe.rsquared * 100:.3} %" for pipe in pipes],
        rmse=[f"{pipe.rmse:.3}" for pipe in pipes],
    )
)

comparison

,tool,memory,feature_complexity,rsquared,rmse
0,getML: RelMT,7d,complex,63.1 %,57.5
1,getML: RelMT,1d,complex,49.0 %,66.9
2,getML: FastProp,7d,simple,56.2 %,62.6
3,getML: FastProp,1d,simple,54.6 %,63.4
4,featuretools,1d,simple,53.7 %,64.3
5,tsfresh,1d,simple,51.0 %,65.9


The summary table shows that combination of both of our hypotheses explains why getML outperforms featuretools and tsfresh. Complex features do better than simple features with a memory of one day. With a memory of seven days, simple features actually get worse. But when you look back seven days and allow more complex features, you get good results.

This suggests that getML outperforms featuretools and tsfresh, because it can make more efficient use of memory and thus look back further. Because RelMT uses feature learning and can build more complex features it can make better use of the greater look-back window.

## 4. Conclusion

We have compared getML's feature learning algorithms to tsfresh's brute-force feature engineering approaches on a data set related to air pollution in China. We found that getML significantly outperforms featuretools and tsfresh. These results are consistent with the view that feature learning can yield significant improvements over simple propositionalization approaches.

However, there are other datasets on which simple propositionalization performs well. Our suggestion is therefore to think of algorithms like `FastProp` and `RelMT` as tools in a toolbox. If a simple tool like `FastProp` gets the job done, then use that. But when you need more advanced approaches, like `RelMT`, you should have them at your disposal as well.

You are encouraged to reproduce these results.

# Next Steps

If you are interested in further real-world applications of getML, visit the [notebook section on getml.com](https://notebooks.getml.com/). If you want to gain a deeper understanding about our notebooks' contents or download the code behind the notebooks, have a look at the [getml-demo repository](https://github.com/getml/getml-demo/). Here, you can also find [futher benchmarks of getML](https://github.com/getml/getml-demo/#benchmarks).

Want to try out getML without much hassle? Just head to [try.getml.com](https://try.getml.com) to launch an instance of getML directly in your browser.

Further, here is some additional material from our [documentation](https://docs.getml.com/latest/) if you want to learn more about getML:
* [Annotating data within getML's data frames](https://docs.getml.com/latest/user_guide/annotating_data/annotating_data.html),
* [Defining your relational structure through getML's abstract data model](https://docs.getml.com/latest/user_guide/data_model/data_model.html), or
* [An introduction to feature learning](https://docs.getml.com/latest/user_guide/feature_engineering/feature_engineering.html).

# Get in contact

If you have any questions, just write us an [email](https://getml.com/contact/lets-talk/). Prefer a private demo of getML for your team? Just [contact us](https://getml.com/contact/lets-talk/) to arrange an introduction to getML.